In [2]:
# MAKING ASYNC MULTIPROCESS VERSION
import asyncio
from concurrent.futures import ThreadPoolExecutor

from IPython import display

import time
from lib_xml_tree import *
from lib_connection import *
from lib_config_loader import load_config



class CameraManager:
    def __init__(self):
        self.state = "free" # free / calibration / inference / loading_weights / loading parameters 
        self.model = None
        self.mode = "inference"   # inference / calibration

    def get_state(self):
        return self.state

    def load_model(self):
        pass
        #load model and params

    def calib_procedure(self):
        print("waiting for self.state")
        while self.state != "free":
            time.sleep(0.05)
            
        self.state = "busy"
        self.mode = "calibration"
        print("loading model params")
        #self.model.load_state_dict()
        time.sleep(0.05)
        print("calibrating")
        time.sleep(0.05)
        self.state = "free"
        
        print('done')
        return


    def heavy_task_2(self):
        self.state = "task_2"
        count = 0
        while count < 50000000:
            count +=1
            time.sleep(0.001)  
         
        return "xyi"



async def main():
    # Load IP, PORT configuration from config
    config = load_config()
    target_address = config['clients']['camera']['target_address']  
    target_port = config['clients']['camera']['target_port']

    UDPClientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
    server_address =  (target_address, target_port)

    # CREATE CameraManager object
    cm = CameraManager()
    cm.load_model()

    

    # PREPARE CHILDREN SUBROCESS POOL
    loop = asyncio.get_running_loop()
    executor = ThreadPoolExecutor()


    





    # PREPARE TO START LOOP
    telegram = create_xml_fast([{'IPOC': '0'}])
    print('start_mess = :', telegram)
    # init dependant telegram variables
    watchDog_out = '0'
    proc = None
    thread = None
    
    while True:
        ################################################
        # SENDING MESSAGES TO CAMERA SERVER-SUBPROCESS
        ################################################  
        TransmitData = telegram
        SendData(TransmitData, UDPClientSocket, server_address)

        


        ################################################
        # RECEIVING MESSAGES FROM CAMERA SERVER-SUBPROCESS
        ################################################ 
        bytesAddressPair = UDPClientSocket.recvfrom(4096)

        ReceivedMessage = bytesAddressPair[0]
        address = bytesAddressPair[1]
        # clientMsg = "Message from Client:{}".format(ReceivedMessage)
        # clientIP = "Client IP Address:{}".format(address)
        #print(clientMsg)
        #print(clientIP)
        received_dict = extract_xml(ReceivedMessage)
        ##print('camera_received_telegram', ReceivedMessage)

        # use received data from cam
        try:
            watchDog_out = received_dict['WatchDog_out']
        except Exception as ex:
            print(f" Data can not be extracted from received to camera telegram: {ex}")   

        ################################################
        # MAIN OPERATION LOGIC
        ################################################ 
        # RUN HEAVY TASK IN BACKGROUND MODE

        # # what to expect in received_dict from robot
        # sent_mess_list.append({"Sen" : {'Type' : 'ServerToCamera'}})    
        # sent_mess_list.append({"WatchDog_out" : watchDog_out})  
        # sent_mess_list.append({'Frame_assigned': frame_assigned})
        # sent_mess_list.append({'Position_reached': position_reached})
        # sent_mess_list.append({'Need_cam_cal': need_cam_cal})
        # sent_mess_list.append({'Screenshot': screenshot}) 

        # run async calib procedure
        #if received_dict['Need_cam_cal'] == '1' and thread = None:
        if watchDog_out and thread == None:
            thread = loop.run_in_executor(executor, cm.calib_procedure)
            print('started')
            

        if thread is not None and thread.done():
            print('finished')
            result = await thread
            thread = None

            
        

        #result = await future
        #print(cm.get_state())


        



        ################################################
        # PREPARING THE MESSAGE TO BE SENT
        ################################################ 
        sent_mess_list = list() # List of dictionaries. every dictionary will be the element of ElementTree
        # sent_mess_list.append({"Sen" : {'Type' : 'ImFree'}})  
        # try:
        sent_mess_list.append({"Sen" : {'Type' : 'Camera'}})   
        sent_mess_list.append({'XYZ1': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ2': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ3': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ4': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'CAM_CAL_RES': {'X': '0.0', 'Y': '0.0', 'Z': '0.0', 'A': '0.0', 'B': '0.0', 'C': '0.0'}})
        sent_mess_list.append({"WatchDog_in" : watchDog_out}) 
        sent_mess_list.append({"Frame_assign" : '0'}) 
        sent_mess_list.append({"Move_next_pt" : '0'}) 
        sent_mess_list.append({"Cam_cal_in_proc" : '0'}) 
        sent_mess_list.append({"See_4_targets" : '0'}) 

        #cresting a new telegram to send
        telegram = create_xml_fast(sent_mess_list)
        ##print('camera_sending_telegram', telegram)
        
                
        #display.clear_output(wait=True)
        await asyncio.sleep(0.001) # DO NOT CHANGE






    
if __name__ == '__main__':
    await main()
    #asyncio.run(main())

        
        

start_mess = : b'<IPOC>0</IPOC>'
waiting for self.state
loading model params
started
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state
loading model params
calibrating
done
finished
started
waiting for self.state


CancelledError: 

In [ ]:
# SYNC VERSION

from IPython import display

import time
from lib_xml_tree import *
from lib_connection import *
from lib_config_loader import load_config


    
if __name__ == '__main__':

    # Load IP, PORT configuration from config
    config = load_config()
    target_address = config['clients']['camera']['target_address']  
    target_port = config['clients']['camera']['target_port']

    UDPClientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
    server_address =  (target_address, target_port)

    
    telegram = create_xml_fast([{'IPOC': '0'}])
    print('start_mess = :', telegram)

    # init dependant telegram variables
    watchDog_out = '0'
    
    while True:
        ################################################
        # SENDING MESSAGES TO CAMERA SERVER-SUBPROCESS
        ################################################  
        TransmitData = telegram
        SendData(TransmitData, UDPClientSocket, server_address)

        


        ################################################
        # RECEIVING MESSAGES FROM CAMERA SERVER-SUBPROCESS
        ################################################ 
        bytesAddressPair = UDPClientSocket.recvfrom(4096)

        ReceivedMessage = bytesAddressPair[0]
        address = bytesAddressPair[1]
        # clientMsg = "Message from Client:{}".format(ReceivedMessage)
        # clientIP = "Client IP Address:{}".format(address)
        #print(clientMsg)
        #print(clientIP)
        received_dict = extract_xml(ReceivedMessage)
        print('camera_received_telegram', ReceivedMessage)

                # use received data from cam
        try:
            watchDog_out = received_dict['WatchDog_out']
        except Exception as ex:
            print(f" Data can not be extracted from received to camera telegram: {ex}")   





        ################################################
        # PREPARING THE MESSAGE TO BE SENT
        ################################################ 
        sent_mess_list = list() # List of dictionaries. every dictionary will be the element of ElementTree
        # sent_mess_list.append({"Sen" : {'Type' : 'ImFree'}})  
        # try:
        sent_mess_list.append({"Sen" : {'Type' : 'Camera'}})   
        sent_mess_list.append({'XYZ1': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ2': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ3': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'XYZ4': {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}})
        sent_mess_list.append({'CAM_CAL_RES': {'X': '0.0', 'Y': '0.0', 'Z': '0.0', 'A': '0.0', 'B': '0.0', 'C': '0.0'}})
        sent_mess_list.append({"WatchDog_in" : watchDog_out}) 
        sent_mess_list.append({"Frame_assign" : '0'}) 
        sent_mess_list.append({"Move_next_pt" : '0'}) 
        sent_mess_list.append({"Cam_cal_in_proc" : '0'}) 
        sent_mess_list.append({"See_4_targets" : '0'}) 

        #cresting a new telegram to send
        telegram = create_xml_fast(sent_mess_list)
        print('camera_sending_telegram', telegram)
        
                
        display.clear_output(wait=True)


        


        
        

In [1]:
import socket
import io
import xml.etree.ElementTree as xml

In [1]:
#for test only 
# Creates structure for ambedding into the xml package, which will be send to robot
def create_Rkorr_for_sending(frame = {'X' : 0, 'Y' : 0, 'Z' : 0}):  #frame = {'X' : float, 'Y' : float, 'Z' : float, 'A' : float, 'B' : float, 'C' : float}
    # print(frame)
    Rkorr = {'X' : str(frame['X']), 
             'Y' : str(frame['Y']), 
             'Z' : str(frame['Z'])}
    
    return Rkorr


def create_xml(target_frame_1 = {'X' : 0.100, 'Y' : 1.000, 'Z' : 0.030, 'A' : 0.000, 'B' : 0.000, 'C' : 0.000},
              target_frame_2 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_3 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_4 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              IPOCtimerReseived = '123456789'):

    SenAttr = dict(Type='ImFree')

    root = xml.Element("Sen", SenAttr)
    
    ################# RESTORE IT ##################
    RkorrArrt_1 = create_Rkorr_for_sending(target_frame_1)
    Rkorr_1 = xml.SubElement(root, "RKorr", RkorrArrt_1)
    ################################################
    #print(RkorrArrt_1)
    #print(Rkorr_1)





    #RkorrArrt = dict(X='0.03')
    #Rkorr = xml.SubElement(root, "RKorr", RkorrArrt)
    #print(RkorrArrt) 
   




    
#     RkorrArrt_2 = create_Rkorr_for_sending(target_frame_2)
#     Rkorr_2 = xml.SubElement(root, "Rkorr", RkorrArrt_2)
    
#     RkorrArrt_3 = create_Rkorr_for_sending(target_frame_3)
#     Rkorr_3 = xml.SubElement(root, "Rkorr", RkorrArrt_3)
    
#     RkorrArrt_4 = create_Rkorr_for_sending(target_frame_4)
#     Rkorr_4 = xml.SubElement(root, "Rkorr", RkorrArrt_4)
    
    
    IPOC = xml.SubElement(root, "IPOC")
    IPOC.text = IPOCtimerReseived
    # IPOC.text = '123456789'
    
    
    tree = xml.ElementTree(root)
    
    
    DecodedTree = xml.tostring(root, encoding='utf8', method='xml', short_empty_elements='true', xml_declaration=False)
    return(DecodedTree)



In [3]:
xml_parcel = create_xml()
xml_parcel

b'<Sen Type="ImFree"><RKorr X="0.1" Y="1.0" Z="0.03" /><IPOC>123456789</IPOC></Sen>'

In [ ]:
d = {'XYZ1' : '123'}
